# Cloud deployment with AWS SageMaker
This notebook showcases how to will **deploy a sentiment analysis model with AWS SageMaker** using serialized pre-trained models. <br> 
- A serialized sklearn TF-IDF vectorizer is used to compute word embeddings 
- A serialized sklearn Logistic Regression to predict polarity (positive or negative).

In [1]:
import os
import json
import joblib
import scipy

In [2]:
!pip install nltk

# 1. Retrieve model artifacts

We will download the pre-trained models from the `ss_deploy_2024` 

In [3]:
# Download pre-trained model and pre-processing function 
# !wget https://raw.githubusercontent.com/laudavid/ss2024_deploy_app/main/streamlit-app/saved_models/logistic_regression.sav # logistic regression model
# !wget https://raw.githubusercontent.com/laudavid/ss2024_deploy_app/main/streamlit-app/saved_models/tfidf-vectorizer.sav # tf-idf vectorizer

# 2. Build an inference script

AWS Sagemaker requires an **inference script** to load the pre-trained model and run predictions. <br>
This script contains the following elements:
-  `model_fn()`: A function that loads the pre-trained models 
-  `predict_fn()`: A function that generates predictions. It includes steps to clean the input data, apply the tf-idf vectorizer and generate predictions with a logistic regression model.
- `input_fn()`/`output_fn()`: Functions for input and output request processing.

*Note: You should always use these function names since SageMaker expects these specific functions to exist when deploying models.*

In [32]:
%%writefile inference.py

import os
import json
import joblib 
from preprocess import text_preprocessing

def model_fn(model_dir):
    """
    Load pre-trained models
    """
    model = joblib.load(os.path.join(model_dir, 'logistic_regression.sav'))
    tfidf = joblib.load(os.path.join(model_dir, 'tfidf-vectorizer.sav'))
    model_dict = {"vectorizer":tfidf, "model":model}
    
    return model_dict


def predict_fn(input_data, model):
    """
    Apply text vectorizer and model to the incoming request
    """
    tfidf = model['vectorizer']
    lr_model = model['model']
        
    clean_text = text_preprocessing(input_data)
    embedding = tfidf.transform(clean_text)
    prediction = lr_model.predict(embedding)

    return prediction.tolist()


def input_fn(request_body, request_content_type):
    """
    Deserialize and prepare the prediction input
    """

    if request_content_type == "application/json":
        request = json.loads(request_body)
    else:
        request = request_body

    return request


def output_fn(prediction, response_content_type):
    """
    Serialize and prepare the prediction output
    """

    if response_content_type == "application/json":
        response = json.dumps(prediction)
    else:
        response = str(prediction) 

    return response

Overwriting inference.py


# 3. Create a requirements.txt file

The `requirements.txt` file allows SageMaker to install the packaged needed for your model to run. <br>
In our case, we added `nltk`as the only external library to install.

*Note: Some third-party libraries such as scikit-learn or pandas are pre-installed in SageMaker and don't need to be added to this file.*

In [33]:
%%writefile requirements.txt 

nltk

Overwriting requirements.txt


# 4. Package and upload to S3
SageMaker requires that the deployment package be structured in a compatible format. <br> It expects all files to be packaged in a tar archive named **"model.tar.gz"** with gzip compression 

We are going to package the pre-trained models, preprocessing and inferencing scripts, as well as the requirements.txt file.

In [34]:
# Create a tarball with the models, scripts, and requirements
!tar -cvpzf model.tar.gz logistic_regression.sav tfidf-vectorizer.sav preprocess.py inference.py requirements.txt

logistic_regression.sav
tfidf-vectorizer.sav
preprocess.py
inference.py
requirements.txt


Now that the model is packaged, we can upload it to an Amazon S3 bucket. <br>
We are going to use `boto3` which is an open-source Python package that allows you to easily interact with other AWS services, in our case Amazon S3 buckets to store the packaged model.

In [35]:
import boto3 # Python package to interact with AWS services (S3,...)
import sagemaker

In [36]:
# Get the S3 bucket created when creating this notebook
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

In [37]:
# Save model to the S3 bucket
prefix = 'sentiment_analysis_lr'

boto3.resource('s3').Bucket(bucket).Object(f'{prefix}/model.tar.gz').upload_file('model.tar.gz')
model_data = f's3://{bucket}/{prefix}/model.tar.gz'

# 5. Deploy the model
We will now use the **SageMaker Python SDK** package to deploy our model to an API endpoint using the package's scikit-learn frameworks. <br> 

In [38]:
import time
from sagemaker.sklearn.model import SKLearnModel

In [39]:
!sudo chmod 777 lost+found # get the right permissions for the lost+found folder for deployment

To deploy the model, you have to **select an instance type** that is in the same region as your session/bucket's region. <br>
To learn more about each region's available instance types, click here: https://aws.amazon.com/fr/sagemaker/pricing/

In [40]:
# Check region of your notebook instance
session = boto3.Session()
region = session.region_name
print(region)

eu-west-3


In [41]:
# Get IAM role created with the notebook
role = sagemaker.get_execution_role()

# Define name of model endpoint (not mandatory)
endpoint_name = "sentiment-analysis-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

**`SKLearnModel`** allows you to build a SageMaker model for deployment using the packaged model <br>
Here are some of its important parameters:
- `model_data`: The packaged model's S3 bucket location 
- `role`: The IAM role to access the S3 bucket 
- `entry_point`: Path of the inference.py file (the file that is executed as the entry point to model hosting)
- `source_dir`: A directory that contains the preprocessing, inference and requirements scripts
- `framework_version`: The scikit-learn package version (it should be the same as the serialized models)
- `py_version`: Python version to execute the model code

For more information, visit SageMaker Python SDK's documentation: https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html

In [43]:
# Build SKLearn SageMaker model
sklearn_model = SKLearnModel(
    model_data=model_data,
    role=role,
    entry_point='inference.py',
    source_dir='.',
    framework_version='1.2-1',
    py_version='py3'
)

# Deploy model to a SageMaker scikit-learn model server
predictor = sklearn_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',
    endpoint_name=endpoint_name
)

-------!

## 6. Test the API endpoint
We can know test the deployed model's API endpoint using `.predict()`. <br>
We specified a JSON serializer and deserialized since SKLearnModel by default has a Numpy serializer and deserializer (which don't fit with our `output_fn` function).

In [48]:
# Test the deployed model
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

data = {'text': "This movie is terrible. I hated the actors and the story."}

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

response = predictor.predict(json.dumps(data))
print(response)

['negative']


# 7. Delete the endpoint

To avoid unnecessary charges, make sure to delete the model and endpoint. <br> 
Don't forget also to stop (or delete) the notebook instance as well as delete the s3 bucket folder with the packaged model.

In [ ]:
# Clean up the model and endpoint 
predictor.delete_model()
predictor.delete_endpoint()